<a href="https://colab.research.google.com/github/ThaoChie/DS-BDS_HN/blob/main/Data_Ceawling/processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy
from collections import defaultdict

In [2]:
bdscom = pd.read_csv("bdscom.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'bdscom.csv'

# Mục mới

In [ ]:
address = pd.read_excel("/Users/thaochie/Documents/hanoiaddress.xlsx")

In [ ]:
address["Quận Huyện"]=address["Quận Huyện"].apply(lambda x: x.replace("Quận ","").strip().lower())
address["Quận Huyện"]=address["Quận Huyện"].apply(lambda x: x.replace("huyện ","").strip().lower())
address["Quận Huyện"]

In [ ]:
address["Xã Phường"]=address["Xã Phường"].apply(lambda x: x.lower().replace("phường ","").strip().lower())
address["Xã Phường"]=address["Xã Phường"].apply(lambda x: x.lower().replace("xã ","").strip().lower())

address["Xã Phường"]

In [ ]:
districts = set(address["Quận Huyện"].tolist())
s = defaultdict(int)
for idx, i in enumerate(bdscom["District"]):
    if i.lower() not in districts and i.lower() not in s:
        print(i)
        if i == "None":
            continue
        s[i.lower()] += 1
        bdscom = bdscom[bdscom["District"] != i]
bdscom.head()

CRAWL GEO COORDINATES

In [ ]:
from geopy.geocoders import Nominatim
import time
from tqdm import tqdm

def get_loc(x, lat=True):
    while True:
        try:
            x.replace("None", "")
            loc = Nominatim(user_agent="GetLoc")
            getLoc = loc.geocode(x)
            if getLoc is None:
                loc = Nominatim(user_agent="GetLoc")
                getLoc = loc.geocode("Hà Nội")
            return (  getLoc.latitude, getLoc.longitude)
        except:
            time.sleep(0.5)


In [ ]:
address = bdscom["Ward"] + " " + bdscom["District"] + " hà nội"

bdscom["X"] = [0 for i in address]
bdscom["Y"] = [0 for i in address]

x = [0 for i in address]
y = [0 for i in address]

In [ ]:
import time
from tqdm import tqdm
import pandas as pd

# Giả sử lookups đã khởi tạo
lookups = {}

for idx, i in enumerate(tqdm(address)):
    try:
        # 1. Kiểm tra dữ liệu đầu vào
        if pd.isna(i) or i == "":
            print(f"\n[SKIP] Dòng {idx} bị rỗng")
            continue

        if i in lookups:
            x[idx] = lookups[i][0]
            y[idx] = lookups[i][1]
        else:
            # 2. Gọi hàm lấy toạ độ
            loc_result = get_loc(i)

            # 3. Kiểm tra kết quả trả về có hợp lệ không
            # Giả sử get_loc trả về [lat, long] hoặc None/[] nếu lỗi
            if loc_result and len(loc_result) >= 2:
                lookups[i] = loc_result
                x[idx] = loc_result[0]
                y[idx] = loc_result[1]
            else:
                print(f"\n[FAIL] Dòng {idx}: Không tìm thấy toạ độ cho '{i}'")
                x[idx] = None
                y[idx] = None

            # 4. QUAN TRỌNG: Ngủ 1 chút để không bị server chặn (1 giây)
            time.sleep(1)

    except Exception as e:
        # 5. Bắt mọi lỗi bất ngờ để vòng lặp không bao giờ chết
        print(f"\n[ERROR] Lỗi tại dòng {idx} ({i}): {e}")
        x[idx] = None
        y[idx] = None

    # Lưu file định kỳ
    if idx % 20 == 0 or idx == len(address) - 1:
        bdscom["X"] = x
        bdscom["Y"] = y
        bdscom.to_csv("final.csv", index=False)